# Sample Code

## 基礎建設

In [270]:
import pandas as pd
import gzip, json

def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

## 載入資料

In [271]:
!wget http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
!wget http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz

--2022-01-02 15:12:40--  http://deepyeti.ucsd.edu/jianmo/amazon/categoryFilesSmall/All_Beauty.csv
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15499476 (15M) [application/octet-stream]
Saving to: ‘All_Beauty.csv.3’

All_Beauty.csv.3    100%[===================>]  14.78M  24.1MB/s    in 0.6s    

2022-01-02 15:12:41 (24.1 MB/s) - ‘All_Beauty.csv.3’ saved [15499476/15499476]

--2022-01-02 15:12:41--  http://deepyeti.ucsd.edu/jianmo/amazon/metaFiles2/meta_All_Beauty.json.gz
Resolving deepyeti.ucsd.edu (deepyeti.ucsd.edu)... 169.228.63.50
Connecting to deepyeti.ucsd.edu (deepyeti.ucsd.edu)|169.228.63.50|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10329961 (9.9M) [application/octet-stream]
Saving to: ‘meta_All_Beauty.json.gz.3’

meta_All_Beauty.jso 100%[===================>]   9.85M  18.7MB/s    in 0.5s   

In [272]:
metadata = getDF('/content/meta_All_Beauty.json.gz')
ratings = pd.read_csv('/content/All_Beauty.csv', names=['asin', 'reviewerID', 'overall', 'unixReviewTime'], header=None)

In [ ]:
ratings.head()

In [ ]:
metadata.shape

# 資料整理

In [273]:
ratings['DATE'] = pd.to_datetime(ratings['unixReviewTime'], unit='s')

# 資料切分

In [275]:
ratings_trainings = ratings[
    (ratings['DATE'] < '2018-09-01')
]
ratings_testings = ratings[
    (ratings['DATE'] >= '2018-09-01') & 
    (ratings['DATE'] <= '2018-09-30')
]
ratings_testings_by_user = ratings_testings.groupby('reviewerID').agg(list).reset_index()[['reviewerID', 'asin']].to_dict('records')
ratings_testings_by_user = { rating['reviewerID']: rating['asin'] for rating in ratings_testings_by_user }
users = list(ratings_testings_by_user.keys())

# 產生需要的欄位

In [276]:
# 產生舊客的 metadata 表

# 處理 rank 中的排行跟分類
metadata['rank_rank'] = metadata['rank'].str.split(" ").str.get(0).str.replace(',','').to_frame()
metadata['rank_category'] = metadata['rank'].str.split("in ").str.get(1).to_frame()


# 將 details 轉為字串
metadata_discover = metadata['details']
tmp = metadata['details']
for i in range(len(metadata_discover)):
  tmp.iloc[i]=json.dumps(metadata_discover.iloc[i]).lower()
metadata['details_str']=tmp


# 抽取需要的欄位
df_asin = metadata['asin'].str.lower().to_frame()
df_title = metadata['title'].str.lower().to_frame()
df_description = metadata['description'].str[0].str.lower().to_frame()
df_brand = metadata['brand'].str.lower().to_frame()
df_rank_category = metadata['rank_category'].str.lower().to_frame()
df_details_str = metadata['details_str'].str.lower().to_frame()


# 產生新的 metadata 表
metadata_tmp = pd.concat([df_asin,df_title,df_description,df_brand,df_rank_category,df_details_str],axis='columns')
metadata_tmp2 = metadata_tmp.set_index('asin')


# 移除重複的資料
metadata_tmp3=metadata_tmp2.drop_duplicates(subset=None, keep='first', inplace=False)
metadata_for_old=metadata_tmp3


In [277]:
# 產生新客的 metadata 表

df_asin = metadata['asin']
df_brand = metadata['brand']
df_title = metadata['title']
df_price = metadata['price']
df_rank = metadata['rank'].str.split(" ").str.get(0).str.replace(',','')
df_rank2 = pd.to_numeric(df_rank, errors='coerce')
df_description = metadata['description']
df_imageURL = metadata['imageURL'].str[0].str.lower().to_frame()
metadata_tmp = pd.concat([df_asin,df_brand,df_title,df_price,df_rank2,df_description,df_imageURL],axis='columns')
metadata_new = metadata_tmp.set_index('asin')
metadata_new['overall_mean'] = ratings.groupby(by=['asin']).mean()['overall']
metadata_new['reviews_count'] = ratings.groupby(by=['asin']).size()


# 抽取商品評分最後日期
ratings_asin=ratings_trainings['asin']
ratings_date=ratings_trainings['DATE']
ratings_tmp = pd.concat([ratings_asin,ratings_date],axis='columns')
ratings_max_dt=ratings_tmp.groupby('asin').DATE.max().to_frame()
ratings_max_dt


metadata_new=pd.merge(metadata_new,ratings_max_dt,how='left', on=['asin'])
metadata_for_new=metadata_new
metadata_for_new

,brand,title,price,rank,description,imageURL,overall_mean,reviews_count,DATE
asin,,,,,,,,,
6546546450,idea village,Loud 'N Clear&trade; Personal Sound Amplifier,,2938573.0,[Loud 'N Clear Personal Sound Amplifier allows...,NaN,2.5,2.0,2013-08-13
7178680776,,No7 Lift &amp; Luminate Triple Action Serum 50...,$44.99,872854.0,[No7 Lift & Luminate Triple Action Serum 50ml ...,NaN,3.0,1.0,2017-10-06
7250468162,No7,No7 Stay Perfect Foundation Cool Vanilla by No7,$28.76,956696.0,[No7 Stay Perfect Foundation now stays perfect...,NaN,5.0,1.0,2017-08-26
7367905066,,Wella Koleston Perfect Hair Colour 44/44 Mediu...,,1870258.0,[],https://images-na.ssl-images-amazon.com/images...,5.0,1.0,2017-04-12
7414204790,Pirmal Healthcare,Lacto Calamine Skin Balance Oil control 120 ml...,$12.15,67701.0,[Lacto Calamine Skin Balance Daily Nourishing ...,https://images-na.ssl-images-amazon.com/images...,4.4,15.0,2018-03-02
...,...,...,...,...,...,...,...,...,...
B01HIWLLUK,,"Barielle Pro Textured Grip Cuticle Nipper, Purple",$9.95,2145325.0,[],NaN,5.0,1.0,2016-09-24
B01HJ1K3YK,Salon Perfect,(Buy 3 Get 1 Free) Salon Perfect Eye Makeup Co...,,1639713.0,[],NaN,1.0,1.0,2017-04-21
B01HJ84SGM,,NOW D-Mannose 500 mg - 120 Veg Capsules (Pack ...,$55.63,207410.0,[],NaN,5.0,5.0,2018-03-23


# 資料探索過程

In [ ]:
## description	also_buy feature also_view	imageURL	imageURLHighRes
metadata_discover=metadata['imageURLHighRes'].str[0].str.lower().to_frame()
aa=metadata_discover.groupby('imageURLHighRes').imageURLHighRes.count().reset_index(name='count').sort_values(['count'], ascending=False)
aa
#aa.iloc[4898].imageURLHighRes
#aa.loc[(aa['count'] == 1)]
#bb=metadata_discover.isnull().sum()
#bb

In [ ]:
##category	tech1	fit	title	tech2	brand	details	main_cat	similar_item	asin	date price rank0 rank1 details_str
metadata_discover=metadata['asin'].str.lower().to_frame()
aa=metadata_discover.groupby('asin').asin.count().reset_index(name='count').sort_values(['count'], ascending=False)
aa
#aa.iloc[2].price
#aa.iloc[3529]
#aa.loc[(aa['count'] == 1)]

In [ ]:
## asin	reviewerID
ratings_discover=ratings['asin'].str.lower().to_frame()
aa=ratings_discover.groupby('asin').asin.count().reset_index(name='count').sort_values(['count'], ascending=False)
aa
#aa.iloc[2].price
#aa.iloc[3529]
#aa.loc[(aa['count'] == 1)]


In [ ]:
ratings_trainings.count()

In [ ]:
ratings_testings.count()

In [ ]:
### asin reviewerID 查看交集狀況
column_asin = ["asin"]
ratings_trainings_item = pd.DataFrame(ratings_trainings['asin'].unique(),columns=column_asin)
ratings_testings_item = pd.DataFrame(ratings_testings['asin'].unique(),columns=column_asin)

ratings_trainings_item = ratings_trainings_item.set_index('asin')
ratings_testings_item = ratings_testings_item.set_index('asin')

#ratings_trainings_item.join(ratings_testings_item, how='inner')
pd.merge(ratings_trainings_item,ratings_testings_item,how='inner', on=['asin'])


# 新客熱門商品

In [1]:
# 優質熱門商品

popular_products = metadata_for_new.loc[(metadata_for_new['overall_mean'] >= 4) & (metadata_for_new['reviews_count'] >= 100) ].sort_values(by=['imageURL','DATE','reviews_count','overall_mean',], ascending=False)
popular_products.reset_index(inplace=True)
popular_products.head(10)
fix_recommend_top10=popular_products['asin'].unique()[0:10].tolist()
fix_recommend_top10

NameError: ignored

## 舊客產生推薦

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# 計算商品用標題所表示的 tfidf 矩陣
df = metadata.drop_duplicates('title')
tf = TfidfVectorizer(analyzer='word')
tfidf_matrix = tf.fit_transform(df['title'])

# 計算商品間的相似程度
from sklearn.metrics.pairwise import cosine_similarity
similarity_matrix = cosine_similarity(tfidf_matrix)
mapping = pd.Series(df.index,index = df['title'])

# 每個商品回傳 k 個最相近的商品
def recommend_item(item_input, k=10):
    try:
        item_index = mapping[item_input]
        similarity_score = list(enumerate(similarity_matrix[item_index]))
        similarity_score = sorted(similarity_score, key=lambda x: x[1], reverse=True)
        similarity_score = similarity_score[:k]
        item_indices = [i[0] for i in similarity_score]
        return (df['asin'].iloc[item_indices].tolist())
    except:
        return []

# 利用使用者購買過的商品產生推薦
def recommend_items(item, k):
    metadata_for_old[metadata_for_old.loc['']==user]['title']
    res = []
    for d in items:
        res.extend(recommend_item(d, k))
    return res


In [300]:
def recommend_rule_and_content(item, k, user):
  
  user_in_trainings = ratings_trainings.loc[(ratings_trainings['reviewerID']==user)]
  
  if user_in_trainings.empty:
    recommend_topk=fix_recommend_top10
    return recommend_topk
  #else:
  #  recommend_items(user, k)


  

In [301]:
def recommender(training_data, users=[], k=10):
    '''
    * training_data: dataframe 輸入的訓練資料集（2018-09-01 以前資料）
    * users: [] 需要被推薦的使用者
    * k: int 每個使用者需要推薦的商品數
    * recommendations: dict
      {
          使用者一： [推薦商品一, 推薦商品二, ...],
          使用者二： [...], ...
      }
    '''
    recommendations = {}
    '''
    content-based
    '''
    ratings_trainings = training_data
    recommendations = {user: recommend_rule_and_content(metadata[metadata['asin'].isin(ratings_trainings[ratings_trainings['reviewerID'] == user]['asin'].tolist())]['title'].tolist(), k, user) for user in users}
    return recommendations

ratings_by_user = recommender(ratings_trainings, users)
ratings_by_user

{'A100XQFWKQ30O2': array(['B000G8LWZI', 'B000FTYALG', 'B01C39X6TW', 'B000WN2T2M',
        'B001V9FPHM', 'B00HY7S61W', 'B018SFIGXG', 'B018SFOZYA',
        'B00021DJ32', 'B001G8LKBS'], dtype=object),
 'A103T1QOGFCSEH': array(['B000G8LWZI', 'B000FTYALG', 'B01C39X6TW', 'B000WN2T2M',
        'B001V9FPHM', 'B00HY7S61W', 'B018SFIGXG', 'B018SFOZYA',
        'B00021DJ32', 'B001G8LKBS'], dtype=object),
 'A106UKKSJ2KXPF': array(['B000G8LWZI', 'B000FTYALG', 'B01C39X6TW', 'B000WN2T2M',
        'B001V9FPHM', 'B00HY7S61W', 'B018SFIGXG', 'B018SFOZYA',
        'B00021DJ32', 'B001G8LKBS'], dtype=object),
 'A10A7GV4D5A11V': array(['B000G8LWZI', 'B000FTYALG', 'B01C39X6TW', 'B000WN2T2M',
        'B001V9FPHM', 'B00HY7S61W', 'B018SFIGXG', 'B018SFOZYA',
        'B00021DJ32', 'B001G8LKBS'], dtype=object),
 'A1119JJ37ZLB8R': array(['B000G8LWZI', 'B000FTYALG', 'B01C39X6TW', 'B000WN2T2M',
        'B001V9FPHM', 'B00HY7S61W', 'B018SFIGXG', 'B018SFOZYA',
        'B00021DJ32', 'B001G8LKBS'], dtype=object),
 'A113UOOL

## 結果評估

In [302]:
def evaluate(ratings_testings_by_user={}, ratings_by_user={}, method=None):
    '''
    * ratings_testings_by_user: dict 真實被購買的商品資料（2018-09-01 以後資料）
    * ratings_by_user: dict 利用訓練資料學習的推薦商品
    * method: str
    * score: float
    '''
    total = 0
    for d in ratings_testings_by_user:
        if d in ratings_by_user:
            total += len(set(ratings_by_user[d]) & set(ratings_testings_by_user[d]))

    score = total / len(ratings_testings)
    return score

evaluate(ratings_testings_by_user, ratings_by_user)

TypeError: ignored